In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Topic: EX2 - Turbofan RUL Prediction
**Task**: Predict the remaining useful life (RUL) of turbofan engines based on given sensor data (time series data). It is a forcasting problem, where the goal is to predict the number of cycles an engine will last before it fails.
**Data**: Turbofan engine degradation simulation data (NASA) - [Link](https://data.nasa.gov/dataset/Turbofan-Engine-Degradation-Simulation-Data-Set/vrks-gjie). See also in the topic [introduction notebook](https://github.com/nina-prog/damage-propagation-modeling/blob/2fb8c1a1102a48d7abbf04e4031807790a913a99/notebooks/Turbofan%20remaining%20useful%20life%20Prediction.ipynb).

**Subtasks**:
1. Perform a deep **exploratory data analysis (EDA)** on the given data.
2. Implement a more efficient **sliding window method** for time series data analysis. -> 🎯 **Focus on this task**
3. Apply **traditional machine learning methods** (SOTA) to predict the remaining useful life. Includes data preparation, feature extraction, feature selection, model selection, and model parameter optimization.
4. Create **neural network models** to predict the remaining useful life. Includes different architectures like Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), or Attention Models. Note: You can search for SOTA research papers and reproduce current state-of-the-art models.


# Imports + Settings

In [3]:
# third-party libraries
import pandas as pd
import numpy as np
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer

import time

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# source code
from src.utils import load_data, load_config, train_val_split_by_group
from src.nn_utils import scale_data, create_sliding_window
from src.nn_util.cnn_models.mainCNNModel import MainCNNModel as CNNModel
from src.nn_util.datamodule.lightning.turbofanDatamodule import TurbofanDatamodule

In [5]:
# settings
sns.set_style("whitegrid")
sns.set_palette("Set2")
sns.set(rc={"figure.dpi":100, 'savefig.dpi':200})
sns.set_context('notebook')

In [6]:
np.random.seed(42)

# Paths

In [11]:
# Make sure to execute this cell only once for one kernel session, before running any other cell below.
os.chdir("../") # set working directory to root of project
os.getcwd() # check current working directory

'C:\\Users\\Johannes\\PycharmProjects\\damage-propagation-modeling'

In [12]:
PATH_TO_CONFIG = "configs/config.yaml"

# Load Config + Data

In [14]:
config = load_config(PATH_TO_CONFIG) # config is dict

In [15]:
train_data, test_data, test_RUL_data = load_data(config_path=PATH_TO_CONFIG, dataset_num=1)

2024-05-24 08:55:59 [src.utils:60] [INFO] >>>> Loading data set 1...
2024-05-24 08:55:59 [src.utils:89] [INFO] >>>> Loaded raw data for dataset 1.
2024-05-24 08:55:59 [src.utils:90] [INFO] >>>> Train Data: (20631, 26)
2024-05-24 08:55:59 [src.utils:91] [INFO] >>>> Test Data: (13096, 26)
2024-05-24 08:55:59 [src.utils:92] [INFO] >>>> Test RUL Data: (100, 1)


# 📍 << Subtask X: TOPIC >>

[TEMPLATE]

Findings:
* Interpretation of plots
* or other key take aways from previous code

In [11]:
# [TEMPLATE] - save processed data (as pickle)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_pickle(f"{config['paths']['processed_data_dir']}ex2_topic_{timestamp}.pkl")

In [12]:
# [TEMPLATE] - save data predictions (as csv)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_csv(f"{config['paths']['prediction_dir']}ex2_topic_{timestamp}.csv", sep=',', decimal='.')

In [13]:
# [TEMPLATE] - save plot results (as png)
fig = plt.figure(figsize=(9, 6))
timestamp = time.strftime("%Y%m%d-%H%M%S")
fig.savefig(f"{config['paths']['plot_dir']}ex2_topic_{timestamp}.png")

<Figure size 900x600 with 0 Axes>

## Only CNN

Inspiration: Paper Dynamic predictive maintenance for multiple components using data-driven
probabilistic RUL prognostics: The case of turbofan engines

In [16]:
window_size = 30

In [17]:
# Setting the seed
pl.seed_everything(17)

Seed set to 17


17

In [18]:
#scoring: Dict[str, make_scorer] = {
#    'mae': make_scorer(mean_absolute_error),
#    'mse': make_scorer(mean_squared_error),
#    'r2': make_scorer(r2_score)
#}

In [19]:
train_data = scale_data(train_data)
test_data = scale_data(test_data)

In [20]:
# TODO: Remove and use Ninas rolling window
# Add RUL to train datasets
from src.rolling_window_creator import calculate_RUL

time_column = 'Cycle'
group_column = 'UnitNumber'

train_data = calculate_RUL(train_data, time_column, group_column)

In [21]:
test_list = []
for engine_number in test_data["UnitNumber"].unique():
    rows_of_engine = test_data["UnitNumber"] == engine_number
    num_rows_of_engine = rows_of_engine.sum()
    remove_first = num_rows_of_engine - window_size
    engine_dataframe = test_data[rows_of_engine][test_data[rows_of_engine]['Cycle'] > remove_first]
    engine_dataframe = engine_dataframe.drop(columns=["UnitNumber", "Cycle"])
    test_list.append(engine_dataframe.values)
    
X_test = np.array(test_list)
y_test = test_RUL_data.values

In [22]:
train, val = train_val_split_by_group(train_data)

2024-05-24 08:56:12 [src.utils:131] [INFO] >>>> Train set contains 82 different engines --> in total 16807
2024-05-24 08:56:12 [src.utils:132] [INFO] >>>>  Test set contains 18 different engines --> in total 3824


In [23]:
X_train, y_train = create_sliding_window(train, window_size=window_size)
X_val, y_val = create_sliding_window(val, window_size=window_size)

Change types of arrays and swap axes:

In [24]:
print(X_train.shape)
X_train = np.swapaxes(X_train, 1, 2)
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
print(X_train.shape)

print(X_val.shape)
X_val = np.swapaxes(X_val, 1, 2)
X_val = np.array(X_val, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)
print(X_val.shape)

print(X_test.shape)
X_test = np.swapaxes(X_test, 1, 2)
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)
print(X_test.shape)

(14429, 30, 24)
(14429, 24, 30)
(3302, 30, 24)
(3302, 24, 30)
(100, 30, 24)
(100, 24, 30)


In [25]:
# Select hyperparameters of trainer!
trainer = Trainer(min_epochs=1, max_epochs=150)
datamodule = TurbofanDatamodule()
datamodule.set_train_dataset(X_train, y_train)
datamodule.set_val_dataset(X_val, y_val)
datamodule.set_predict_dataset(X_test)
datamodule.set_test_dataset(X_test, y_test)
model = CNNModel(lr=0.0001, window_size=window_size, dropout_rate=0.2)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [24]:
%%capture
trainer.fit(model, datamodule=datamodule)


  | Name        | Type        | Params
--------------------------------------------
0 | loss        | MSELoss     | 0     
1 | dropout     | Dropout     | 0     
2 | layer1_conv | Conv1d      | 4.8 K 
3 | layer1_bn   | BatchNorm2d | 80    
4 | layer2_conv | Conv1d      | 8.0 K 
5 | fc1         | Linear      | 56.4 K
6 | fc2         | Linear      | 4.2 K 
7 | fc3         | Linear      | 65    
--------------------------------------------
73.6 K    Trainable params
0         Non-trainable params
73.6 K    Total params
0.294     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=150` reached.


In [119]:
pred = trainer.test(model, datamodule=datamodule, ckpt_path='best')

Restoring states from the checkpoint path at C:\Users\Johannes\PycharmProjects\lightning_logs\version_7\checkpoints\epoch=149-step=33900.ckpt
Loaded model weights from the checkpoint at C:\Users\Johannes\PycharmProjects\lightning_logs\version_7\checkpoints\epoch=149-step=33900.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

tensor(2423.4902)
tensor(3135.9062)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │      2679.9599609375      │
└───────────────────────────┴───────────────────────────┘